<a href="https://colab.research.google.com/github/joao-dias-25/dataeng-spark/blob/main/spark_streaming/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
    - extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [2]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.6 MB/s eta 0:00:00


# Producer

In [183]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.option('checkpointLocation', 'content/lake/bronze/messages/checkpoint')
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)

query.awaitTermination(60)


False

In [184]:
query.stop()

In [185]:
#!rm -rf content/lake/bronze/messages

In [186]:

df = spark.read.format("parquet").load("content/lake/bronze/messages/data")
df.printSchema()
df.show()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)
 |-- event_type: string (nullable = true)
 |-- message_id: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- country_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-23 12:14:...| 6306|  RECEIVED|bd5c47e1-c137-469...|  EMAIL|      2001|   1043|
|2024-12-23 12:14:...| 6308|  RECEIVED|bd5c47e1-c137-469...|  EMAIL|      2001|   1043|
|2024-12-23 12:14:...| 6310|  RECEIVED|bd5c47e1-c137-469...|  EMAIL|      2001|   1043|
|2024-12-23 12:14:...| 6312|  RECEIVED|bd5c47e1-c137-469...|  EMAIL|      2001|   1043|
|2024-12-23 12:14:...| 6307|  RECEIVED|bd5c47e1-c137-469...|  EMAIL|      2001|   1043|
|2024

# Additional datasets

In [187]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [201]:
# TODO
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col, date_format

rate_schema = StructType([
StructField('timestamp', TimestampType(),True),
StructField('value',LongType(),True),
StructField('event_type',StringType(),True),
StructField('message_id',StringType(),True),
StructField('channel', StringType(), True),
StructField('country_id', IntegerType(), True),
StructField('user_id', IntegerType(), True),
])

stream2 = spark.readStream.format('parquet').schema(rate_schema).load('content/lake/bronze/messages/data')

stream_not_null =stream2.filter((col("event_type").isNotNull()) &
        (col("event_type") != '') &
        (col("event_type") != 'NONE'))
stream_null = stream2.filter((F.col("event_type").isNull()) |
        (col("event_type") == '') |
        (col("event_type") == 'NONE'))

# Check if dataframe is streaming
print(stream2.isStreaming)
print(stream_null.isStreaming)
print(stream_not_null.isStreaming)

def save_parquet_c(df, batch_id):
  (df
    .withColumn('date', date_format('timestamp',"yyyyMMdd"))
    .join(countries, "country_id", "left")
    .write.mode("append")
    .partitionBy("date")
    .parquet('content/lake/silver/messages_corrupted/data'))

def save_parquet(df, batch_id):
  (df
    .withColumn('date', date_format('timestamp',"yyyyMMdd"))
    .join(countries, "country_id", "left")
    .write.mode("append")
    .partitionBy("date")
    .parquet('content/lake/silver/messages/data'))


  # Start write as streaming into parquet
query = (stream_null.writeStream
  .option('checkpointLocation', 'content/lake/silver/messages_corrupted/checkpoint')
  .trigger(processingTime='5 seconds')
  .outputMode('append')
  .foreachBatch(save_parquet_c)
  .start() )

query2 = (stream_not_null.writeStream
  .option('checkpointLocation', 'content/lake/silver/messages/checkpoint')
  .trigger(processingTime='5 seconds')
  .outputMode('append')
  .foreachBatch(save_parquet)
  .start() )

query.awaitTermination(20)
query2.awaitTermination(20)

True
True
True


False

In [202]:
query.stop()
query2.stop()


In [203]:
#!rm -rf content/lake/silver
#!ls content/lake/bronze/messages/data

## Checking data

In [204]:
# TODO



df = spark.read.format("parquet").load("content/lake/silver/messages/data")
df.printSchema()
df.show()

root
 |-- country_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)
 |-- event_type: string (nullable = true)
 |-- message_id: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- date: integer (nullable = true)

+----------+--------------------+-----+----------+--------------------+-------+-------+--------+--------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id| country|    date|
+----------+--------------------+-----+----------+--------------------+-------+-------+--------+--------+
|      2000|2024-12-23 12:14:...|   24|      OPEN|8f389275-6f54-47e...|   CHAT|   1035|  Brazil|20241223|
|      2000|2024-12-23 12:14:...|   26|      OPEN|8f389275-6f54-47e...|   CHAT|   1035|  Brazil|20241223|
|      2000|2024-12-23 12:14:...|   28|      OPEN|8f389275-6f54-47e...|   CHAT|   1035|  Brazil|2024122

In [205]:

df_notcorrupted = spark.read.format("parquet").load("content/lake/silver/messages/data")
df_corrupted = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data")
df_source = spark.read.format("parquet").load("content/lake/bronze/messages/data")

print(f"not corrupted - {df_notcorrupted.count()}")
print(f"corrupted  - {df_corrupted.count()}")
print(f"source - {df_source.count()}")

not corrupted - 52
corrupted  - 9
source - 61


# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [197]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages/data")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [198]:
# report 1
# TODO

from pyspark.sql.functions import *

#dedup.show()
aggregated_data = dedup.groupBy("date", "channel", "event_type").agg(
    count("message_id").alias("message_count")
)


pivoted_data = aggregated_data.groupBy("date", "channel").pivot("event_type").agg(first("message_count"))


pivoted_data.show()


+--------+-------+-------+-------+----+--------+----+
|    date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+--------+-------+-------+-------+----+--------+----+
|20241223|    SMS|     43|     38|  40|      44|  41|
|20241223|   CHAT|     44|     47|  45|      45|  43|
|20241223|  OTHER|     43|     45|  39|      48|  45|
|20241223|  EMAIL|     39|     45|  40|      44|  40|
|20241223|   PUSH|     46|     41|  43|      47|  42|
+--------+-------+-------+-------+----+--------+----+



## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [199]:
# report 2
# TODO
dedup.show()


user_activity = dedup.groupBy("user_id", "channel").agg(
    F.count("message_id").alias("message_count")
)

# user total
user_total = user_activity.groupBy("user_id").agg(
    F.sum("message_count").alias("iterations")
)

# user by channel
user_by_channel= user_activity.groupBy("user_id").pivot("channel", ["CHAT", "EMAIL", "OTHER", "PUSH", "SMS"]).agg(F.first("message_count"))


final_report = user_total.join(user_by_channel, on="user_id", how="left")


final_report.show()

+----------+--------------------+-----+----------+--------------------+-------+-------+--------------+--------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|       country|    date|
+----------+--------------------+-----+----------+--------------------+-------+-------+--------------+--------+
|      2007|2024-12-23 10:58:...| 1747|   CLICKED|02f6520a-2493-4ab...|   CHAT|   1046| United States|20241223|
|      2006|2024-12-23 11:04:...| 2137|   CLICKED|02f6520a-2493-4ab...|  OTHER|   1009|United Kingdom|20241223|
|      2013|2024-12-23 10:31:...|  152|   CLICKED|02f6520a-2493-4ab...|   PUSH|   1038|   South Korea|20241223|
|      2005|2024-12-23 10:43:...|  867|   CREATED|02f6520a-2493-4ab...|   CHAT|   1031|         Italy|20241223|
|      2005|2024-12-23 11:16:...| 2829|   CREATED|02f6520a-2493-4ab...|  EMAIL|   1007|         Italy|20241223|
|      2003|2024-12-23 10:50:...| 1270|   CREATED|02f6520a-2493-4ab...|  OTHER|   1025|       Germany|20

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

You could use spark structure Streaming consuming date from kafka using Windowed Aggregations for agregating data and stateful processing to be used to track and update aggregates as new messages arrive

- Which storage would you use and why? (database?, data lake?, kafka?)


because the data will be consumed by a dashboard, it requires a fast query engine, which could be a real time database or a data warehouse like bigquery
